Version 1.1.0

# Mean encodings

In this programming assignment you will be working with `1C` dataset from the final competition. You are asked to encode `item_id` in 4 different ways:

    1) Via KFold scheme;  
    2) Via Leave-one-out scheme;
    3) Via smoothing scheme;
    4) Via expanding mean scheme.

**You will need to submit** the correlation coefficient between resulting encoding and target variable up to 4 decimal places.

### General tips

* Fill NANs in the encoding with `0.3343`.
* Some encoding schemes depend on sorting order, so in order to avoid confusion, please use the following code snippet to construct the data frame. This snippet also implements mean encoding without regularization.

In [1]:
import pandas as pd
import numpy as np
from itertools import product
from grader import Grader

# Read data

In [2]:
sales = pd.read_csv('C:\\Work\\Python\\competitive-data-science\\final-project\\sales_train.csv.gz')

In [3]:
print(sales.shape)
sales.head()

(2935849, 6)


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


# Aggregate data

Since the competition task is to make a monthly prediction, we need to aggregate the data to montly level before doing any encodings. The following code-cell serves just that purpose.

In [4]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})

#fix column names
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

c:\users\italo dell\appdata\local\programs\python\python35\lib\site-packages\pandas\core\groupby.py:4189: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [5]:
gb.head()
all_data.head()

,shop_id,item_id,date_block_num,target
139255,0,19,0,0.0
141495,0,27,0,0.0
144968,0,28,0,0.0
142661,0,29,0,0.0
138947,0,32,0,6.0


# Mean encodings without regularization

After we did the techinical work, we are ready to actually *mean encode* the desired `item_id` variable. 

Here are two ways to implement mean encoding features *without* any regularization. You can use this code as a starting point to implement regularized techniques. 

#### Method 1

In [6]:
# Calculate a mapping: {item_id: target_mean}
item_id_target_mean = all_data.groupby('item_id').target.mean()

# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.483038698862


In [7]:
item_id_target_mean

item_id
0        0.020000
1        0.023810
2        0.019802
3        0.019802
4        0.020000
5        0.020000
6        0.020000
7        0.020000
8        0.019802
9        0.019608
10       0.020000
11       0.020000
12       0.021739
13       0.020000
14       0.020000
15       0.020000
16       0.020000
17       0.020000
18       0.019608
19       0.022222
20       0.019608
21       0.020000
22       0.021277
23       0.021277
24       0.021277
25       0.019608
26       0.019231
27       0.056834
28       0.141176
29       0.037383
           ...   
22140    0.191461
22141    0.276042
22142    0.069930
22143    1.930804
22144    0.266667
22145    0.650407
22146    0.040334
22147    0.082100
22148    0.021739
22149    0.058923
22150    0.088821
22151    1.159375
22152    0.155109
22153    0.026627
22154    0.109870
22155    0.093671
22156    0.029197
22157    0.021978
22158    0.022727
22159    0.172414
22160    0.097030
22161    0.022222
22162    1.556793
22163    0.581395
22

#### Method 2

In [8]:
'''
     Differently to `.target.mean()` function `transform` 
   will return a dataframe with an index like in `all_data`.
   Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform('mean')

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.483038698862


See the printed value? It is the correlation coefficient between the target variable and your new encoded feature. You need to **compute correlation coefficient** between the encodings, that you will implement and **submit those to coursera**.

In [9]:
grader = Grader()

In [10]:
all_data.head()

,shop_id,item_id,date_block_num,target,item_target_enc
139255,0,19,0,0.0,0.022222
141495,0,27,0,0.0,0.056834
144968,0,28,0,0.0,0.141176
142661,0,29,0,0.0,0.037383
138947,0,32,0,6.0,1.319042


# 1. KFold scheme

Explained starting at 41 sec of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

**Now it's your turn to write the code!** 

You may use 'Regularization' video as a reference for all further tasks.

First, implement KFold scheme with five folds. Use KFold(5) from sklearn.model_selection. 

1. Split your data in 5 folds with `sklearn.model_selection.KFold` with `shuffle=False` argument.
2. Iterate through folds: use all but the current fold to calculate mean target for each level `item_id`, and  fill the current fold.

    *  See the **Method 1** from the example implementation. In particular learn what `map` and pd.Series.map functions do. They are pretty handy in many situations.

In [11]:
# YOUR CODE GOES HERE
from sklearn.model_selection import KFold
y_tr = all_data['target'].values
kf = KFold(5)
cols= all_data.item_id.unique()
i=0
t = all_data.copy()
t['item_target_enc_cv'] = np.nan
#t['item_target_enc_reg'] = np.nan
for ind_tr, ind_val in kf.split(all_data):
    X_tr,X_val = all_data.iloc[ind_tr], all_data.iloc[ind_val]
    means = X_val['item_id'].map(X_tr.groupby('item_id').target.mean())
    X_val['item_target_enc_cv'] = means
    t.iloc[ind_val] = X_val

print(t.columns)    

t['item_target_enc_cv'].fillna(0.3343, inplace=True) 

print(t.item_target_enc_cv.mean())



c:\users\italo dell\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


Index(['shop_id', 'item_id', 'date_block_num', 'target', 'item_target_enc',
       'item_target_enc_cv'],
      dtype='object')
0.351480561248


In [12]:
encoded_feature = t['item_target_enc_cv'].values
# You will need to compute correlation like that
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('KFold_scheme', corr)

0.41645907128
Current answer for task KFold_scheme is: 0.41645907128


# 2. Leave-one-out scheme

Now, implement leave-one-out scheme. Note that if you just simply set the number of folds to the number of samples and run the code from the **KFold scheme**, you will probably wait for a very long time. 

To implement a faster version, note, that to calculate mean target value using all the objects but one *given object*, you can:

1. Calculate sum of the target values using all the objects.
2. Then subtract the target of the *given object* and divide the resulting value by `n_objects - 1`. 

Note that you do not need to perform `1.` for every object. And `2.` can be implemented without any `for` loop.

It is the most convenient to use `.transform` function as in **Method 2**.

In [28]:
t = all_data.copy()
soma = all_data.groupby('item_id')['target'].transform('sum')
contagem = all_data.groupby('item_id')['target'].transform('count')
t['mean_target_loo'] = (soma - t['target']) / (contagem - 1)


In [29]:
print(t[t.item_id == 19])
print(t.iloc[139255])
print(t.iloc[147370])
print(t.iloc[9415])

        shop_id  item_id  date_block_num  target  item_target_enc  \
139255        0       19               0     0.0         0.022222   
147370        1       19               0     0.0         0.022222   
114910        2       19               0     0.0         0.022222   
123025        3       19               0     0.0         0.022222   
98680         4       19               0     0.0         0.022222   
106795        6       19               0     0.0         0.022222   
131140        7       19               0     0.0         0.022222   
171715        8       19               0     0.0         0.022222   
179830       10       19               0     0.0         0.022222   
204175       12       19               0     0.0         0.022222   
196060       13       19               0     0.0         0.022222   
187945       14       19               0     0.0         0.022222   
163600       15       19               0     0.0         0.022222   
155485       16       19          

In [30]:
# YOUR CODE GOES HERE
encoded_feature = t.mean_target_loo.values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Leave-one-out_scheme', corr)

0.480384831129
Current answer for task Leave-one-out_scheme is: 0.480384831129


In [ ]:
t.head(25)

# 3. Smoothing

Explained starting at 4:03 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Next, implement smoothing scheme with $\alpha = 100$. Use the formula from the first slide in the video and $0.3343$ as `globalmean`. Note that `nrows` is the number of objects that belong to a certain category (not the number of rows in the dataset).

In [34]:
t = all_data.copy()
t['mean_target_smt'] = (t.groupby('item_id')['target'].transform('mean') *
                 t.groupby('item_id')['target'].transform('count') + 0.3343 * 100 ) / (t.groupby('item_id')['target'].transform('count') + 100)
t.head()

,shop_id,item_id,date_block_num,target,item_target_enc,mean_target_smt
139255,0,19,0,0.0,0.022222,0.237448
141495,0,27,0,0.0,0.056834,0.089905
144968,0,28,0,0.0,0.141176,0.168964
142661,0,29,0,0.0,0.037383,0.107910
138947,0,32,0,6.0,1.319042,1.260635


In [36]:
# YOUR CODE GOES HERE

encoded_feature = t.mean_target_smt.values

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Smoothing_scheme', corr)

0.48181987971
Current answer for task Smoothing_scheme is: 0.48181987971


# 4. Expanding mean scheme

Explained starting at 5:50 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Finally, implement the *expanding mean* scheme. It is basically already implemented for you in the video, but you can challenge yourself and try to implement it yourself. You will need [`cumsum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.cumsum.html) and [`cumcount`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html) functions from pandas.

In [62]:
t = all_data.copy()
t['item_id_mean_target_ms'] = (t.groupby('item_id')['target'].cumsum() - t.target)/(t.groupby('item_id').cumcount()+1)

In [63]:
# YOUR CODE GOES HERE
encoded_feature = t['item_id_mean_target_ms'].values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Expanding_mean_scheme', corr)

0.501101555985
Current answer for task Expanding_mean_scheme is: 0.501101555985


## Authorization & Submission
To submit assignment parts to Cousera platform, please, enter your e-mail and token into variables below. You can generate token on this programming assignment page. Note: Token expires 30 minutes after generation.

In [66]:
STUDENT_EMAIL = 'itscefet03@yahoo.com.br'
STUDENT_TOKEN = '7Vnj06tHXYqCctML'
grader.status()

You want to submit these numbers:
Task KFold_scheme: 0.41645907128
Task Leave-one-out_scheme: 0.480384831129
Task Smoothing_scheme: 0.48181987971
Task Expanding_mean_scheme: 0.501101555985


In [67]:
grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)

Submitted to Coursera platform. See results on assignment page!
